In [18]:
import gspread
from google.oauth2.service_account import Credentials

SERVICE_ACCOUNT_FILE = r"C:\Users\Swarupa\Downloads\sheetsconnect-477105-1c2fc805323b.json"

scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scope)
client = gspread.authorize(creds)

SPREADSHEET_ID = "1obey-A5qj8hKRG333Oi43R6uEVeJa7TiNtjtfLpIP1o"

sh = client.open_by_key(SPREADSHEET_ID)
print("OK - Connected\nSpreadsheet Title:", sh.title)

ws = sh.sheet1
print("First sheet title:", ws.title)


OK - Connected
Spreadsheet Title: CombiFinal
First sheet title: Sheet1


In [30]:
import pandas as pd
import itertools
import math
import gc
from tqdm import tqdm
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

# === USER SETTINGS ===
INPUT_FILE = r"C:\Users\Swarupa\Downloads\50 comp + EURINR Value + Nifty 50 index.xlsx"
SERVICE_ACCOUNT_FILE = r"C:\Users\Swarupa\Downloads\sheetsconnect-477105-1c2fc805323b.json"
SPREADSHEET_ID = "1obey-A5qj8hKRG333Oi43R6uEVeJa7TiNtjtfLpIP1o"

# sheet names
SHEET_ALL = "ALL"
SHEET_EURINR = "EURINR"
SHEET_NIFTY = "NIFTY"
SHEET_COMMON = "COMMON"

ROLL_WINDOW = 50
QUANTITY_CAPITAL = 200000.0

# connect
scope = [ "https://www.googleapis.com/auth/spreadsheets",
          "https://www.googleapis.com/auth/drive"
]
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scope)
client = gspread.authorize(creds)

sh = client.open_by_key(SPREADSHEET_ID)
print("google sheet connection ok")


google sheet connection ok


In [31]:
print("loading Excel...")
df_head = pd.read_excel(INPUT_FILE, nrows=1)
cols = list(df_head.columns)
print(cols)


loading Excel...
['DATE', 'Adani ent', 'Adani Port', 'Apollo Hosp', 'Asian Paint', 'Axis bank', 'Bajaj Auto', 'Bajaj fins', 'Bajaj Finance', 'BEL', 'Bharti Airtel', 'Cipla', 'Coal india', 'Dr reddy', 'Eicher Mot', 'Eternal', 'Grasim', 'HCL Tech', 'Hdfcbank', 'HDFC life', 'Heromoto', 'Hindalco', 'HUL', 'ICICIBANK', 'Indusind', 'Infy', 'ITC', 'Jiofin', 'JSW Steel', 'Kotak Bank', 'LT', 'M&M', 'Maruti', 'Nestle', 'NTPC', 'ONGC', 'Powergrid', 'Reliance', 'SBI Life', 'SBIN', 'Shriram Fin', 'Sun Pharma', 'Tata cons', 'Tata motors', 'Tata steel', 'TCS', 'Techm', 'Titan', 'Trent', 'Ultratech', 'Wipro', 'EURINR', 'Nifty Index']


In [32]:
df = pd.read_excel(INPUT_FILE)
df = df.sort_values(by=df.columns[0]).reset_index(drop=True)
df.rename(columns={df.columns[0]:"Date"}, inplace=True)
print(df.head())


        Date  Adani ent  Adani Port  Apollo Hosp  Asian Paint  Axis bank  \
0 2020-01-01     207.85      377.65      1426.35      1793.20     748.70   
1 2020-01-02     211.20      383.15      1494.65      1790.65     756.95   
2 2020-01-03     208.30      382.50      1486.10      1751.40     742.95   
3 2020-01-06     199.55      380.20      1462.50      1707.15     723.25   
4 2020-01-07     204.05      384.80      1478.70      1724.40     725.75   

   Bajaj Auto  Bajaj fins  Bajaj Finance    BEL  ...  Tata motors  Tata steel  \
0     3150.10      937.84         423.13  33.37  ...       184.45       46.77   
1     3121.00      950.56         424.60  34.37  ...       193.75       48.48   
2     3072.05      934.58         419.34  33.64  ...       191.10       48.37   
3     3037.65      904.27         399.67  32.31  ...       185.65       47.32   
4     3037.95      909.56         400.76  32.11  ...       184.70       47.61   

       TCS   Techm    Titan   Trent  Ultratech   Wipro  

In [34]:
# these 2 macro columns must be last two
EUR = "EURINR"
NIFTY = "Nifty Index"

# detect all columns that are NOT date and not macro
company_cols = [c for c in df.columns if c not in ["Date", EUR, NIFTY]]

print("company columns count =", len(company_cols))
print(company_cols[:10], "...", company_cols[-3:])

df_comp = df[["Date"] + company_cols]
df_comp.head()



company columns count = 50
['Adani ent', 'Adani Port', 'Apollo Hosp', 'Asian Paint', 'Axis bank', 'Bajaj Auto', 'Bajaj fins', 'Bajaj Finance', 'BEL', 'Bharti Airtel'] ... ['Trent', 'Ultratech', 'Wipro']


,Date,Adani ent,Adani Port,Apollo Hosp,Asian Paint,Axis bank,Bajaj Auto,Bajaj fins,Bajaj Finance,BEL,...,Sun Pharma,Tata cons,Tata motors,Tata steel,TCS,Techm,Titan,Trent,Ultratech,Wipro
0,2020-01-01,207.85,377.65,1426.35,1793.20,748.70,3150.10,937.84,423.13,33.37,...,434.30,319.86,184.45,46.77,2167.60,762.10,1154.75,529.65,4065.0,123.85
1,2020-01-02,211.20,383.15,1494.65,1790.65,756.95,3121.00,950.56,424.60,34.37,...,434.95,319.37,193.75,48.48,2157.65,766.05,1155.60,541.35,4244.8,124.15
2,2020-01-03,208.30,382.50,1486.10,1751.40,742.95,3072.05,934.58,419.34,33.64,...,444.60,314.28,191.10,48.37,2200.65,775.10,1139.70,535.90,4219.2,125.55
3,2020-01-06,199.55,380.20,1462.50,1707.15,723.25,3037.65,904.27,399.67,32.31,...,439.95,308.26,185.65,47.32,2200.45,770.40,1158.60,529.95,4157.1,126.07
4,2020-01-07,204.05,384.80,1478.70,1724.40,725.75,3037.95,909.56,400.76,32.11,...,446.40,311.02,184.70,47.61,2205.85,777.10,1159.95,532.95,4242.1,127.60


In [39]:
ws.update('A1', [["OK"]])
print("written")


C:\Users\Swarupa\AppData\Local\Temp\ipykernel_14000\2457633785.py:1: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  ws.update('A1', [["OK"]])


written


In [41]:
df_comp["Date"] = df_comp["Date"].astype(str)


C:\Users\Swarupa\AppData\Local\Temp\ipykernel_14000\1171101270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comp["Date"] = df_comp["Date"].astype(str)


In [43]:
df_comp["Date"] = df_comp["Date"].astype(str)
df_comp = df_comp.fillna("")        # remove NaN
values = [df_comp.columns.tolist()] + df_comp.values.tolist()


C:\Users\Swarupa\AppData\Local\Temp\ipykernel_14000\2665974246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comp["Date"] = df_comp["Date"].astype(str)


In [44]:
ws.update(range_name='A1', values=values)
print("OK uploaded")


OK uploaded


In [45]:
# ---- FINAL UPLOAD ----

# convert date to string
df_comp["Date"] = df_comp["Date"].astype(str)

# replace NaN -> blank
df_comp = df_comp.fillna("")

# make values
values = [df_comp.columns.tolist()] + df_comp.values.tolist()

# clear the sheet completely
ws.clear()

# upload from A1
ws.update(range_name='A1', values=values)

print("FULL DATA WRITTEN")


FULL DATA WRITTEN
